## 1. 뉴스제목 가져오기
* user-agent 요청헤더를 반드시 설정해야 한다.

In [4]:
# requests 라이브러리 설치여부 확인
!pip show requests

Name: requests
Version: 2.32.3
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: C:\Users\user\anaconda3\Lib\site-packages
Requires: certifi, charset-normalizer, idna, urllib3
Required-by: anaconda-catalogs, anaconda-client, anaconda-cloud-auth, anaconda-project, conda, conda-build, conda-repo-cli, conda_package_streaming, cookiecutter, datashader, jupyterlab_server, panel, requests-file, requests-toolbelt, Sphinx, streamlit, tldextract


In [5]:
# beautifulsoup4 라이브러리 설치여부 확인
!pip show beautifulsoup4

Name: beautifulsoup4
Version: 4.12.3
Summary: Screen-scraping library
Home-page: https://www.crummy.com/software/BeautifulSoup/bs4/
Author: 
Author-email: Leonard Richardson <leonardr@segfault.org>
License: MIT License
Location: C:\Users\user\anaconda3\Lib\site-packages
Requires: soupsieve
Required-by: conda-build, nbconvert


In [6]:
# reqeusts, bs4 import
import requests 
# BeautifulSoup import
import bs4
#beautifulsoup 클래스 import
from bs4 import BeautifulSoup

In [7]:
# requests, bs4 버전 확인하기
print(f'requests 버전 = {requests.__version__}')
print(f'bs4 버전 = {bs4.__version__}')

requests 버전 = 2.32.3
bs4 버전 = 4.12.3


### 1. 뉴스 제목 추출하기

In [8]:
# IT/과학 뉴스 
req_param = {
    'sid': 105
}

sid = 104
#url =f'https://news.naver.com/section/{sid}' 
url = 'https://news.naver.com/section/{sid}'.format(**req_param)
print(url)


https://news.naver.com/section/105


In [9]:
# IT/과학 뉴스 
req_param = {
    'sid': 105
}
url = 'https://news.naver.com/section/{sid}'.format(**req_param)
print(url)

# 요청 헤더 설정 : 브라우저 정보
req_header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
}

# requests 의 get() 함수 호출하기 
res=requests.get(url,headers=req_header)
print(res.status_code, res.ok)
print(type(res))
print(res)

# 응답(response)이 OK 이면
# 응답 (response)에서 text 추출
if res.ok:
    html = res.text
    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(html, "html.parser")   
    #print(soup)
    # CSS 선택자
    # print(soup.select("div.sa_text a[href*='mnews/article']"))
    '''
       <div class='sa_text'> 
          <a href="https://n.news.naver.com/mnews/article/366/0001094461" class="sa_text_title">
    '''
    a_tags = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/article'].sa_text_title")
    print(len(a_tags))
    print(type(a_tags)) #[Tag,Tag,Tag]
    # <a> 태그 리스트 순회하기
    for idx,a_tag in enumerate(a_tags,1):        
        if a_tag.text:
            title = a_tag.text.strip() # 제목
            link = a_tag['href'] # 링크
            #print(len(title))
            print(f'{idx} {title} {link}')
else:
    # 응답(response)이 Error 이면 status code 출력    
    print(f'Error Code = {res.status_code}')


https://news.naver.com/section/105
200 True
<class 'requests.models.Response'>
<Response [200]>
43
<class 'bs4.element.ResultSet'>
1 솔트룩스, KT ‘믿음’과 손잡아…‘독자 AI’ 사업 참전 https://n.news.naver.com/mnews/article/009/0005529262
2 갤럭시 Z 폴드7·Z 플립7, 폴더블 최다 사전판매 신기록.. 개통 시작 https://n.news.naver.com/mnews/article/014/0005380605
3 네이버클라우드, 140억 파라미터 추론모델도 상업용 무료 공개 https://n.news.naver.com/mnews/article/277/0005626112
4 불법 사이트 수사 피해 日 귀화까지…만화가들 "뉴토끼 운영자 송환 촉구" https://n.news.naver.com/mnews/article/119/0002982265
5 "올해 삼성 폴더블폰 점유율 10%P↓ 전망…내년 애플 출시 관건" https://n.news.naver.com/mnews/article/001/0015522990
6 ‘K-AI휴머노이드’ 개발…출연연·대학·기업 어벤져스 ‘전략연구단’ 떴다 https://n.news.naver.com/mnews/article/016/0002503305
7 게임, K콘텐츠 주역인데…일자리 8만개 사라질 수도 있다고? https://n.news.naver.com/mnews/article/011/0004512256
8 "SKT 위약금 면제로 7조 원 손실? 국민 협박, 사과 안 하면 고발" https://n.news.naver.com/mnews/article/006/0000130992
9 “50% 할인에도 하루 1대도 안팔려” 망할 줄 알았더니… 난리난 ‘접는폰’ 반전 https://n.news.naver.com/mnews/article/016/0002503447
10 한화에어로-항우연, 누

### 1.1 뉴스제목 추출하는 함수 선언하기

In [10]:
import requests
from bs4 import BeautifulSoup

#section_dict = {100:'정치',101:'경제',102:'사회',103:'생활/문화',104:'세계',105:'IT/과학'}
section_dict = {'정치':100,'경제':101,'사회':102,'생활/문화':103,'세계':104,'IT/과학':105}

def print_news(section):  #print_new(103,'생활/문화') 
    sid = section_dict[section]
    url = f'https://news.naver.com/section/{sid}'
    print(f'{url} {section} 뉴스')

    # 요청 헤더 설정 : 브라우저 정보
    req_header = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    }
    # requests 의 get() 함수 호출하기 
    res=requests.get(url,headers=req_header)
    print(res.status_code, res.ok)
    if res.ok:
        html = res.text
        # BeautifulSoup 객체 생성
        soup = BeautifulSoup(html, "html.parser")
        a_tags = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/article'].sa_text_title")
        print(len(a_tags))
        # <a> 태그 리스트 순회하기
        for idx,a_tag in enumerate(a_tags,1):        
            if a_tag.text:
                title = a_tag.text.strip() # 제목
                link = a_tag['href'] # 링크
                #print(len(title))
                print(f'{idx} {title} {link}')
    else:
        # 응답(response)이 Error 이면 status code 출력    
        print(f'Error Code = {res.status_code}')      

   

In [11]:
print_news('경제')

https://news.naver.com/section/101 경제 뉴스
200 True
46
1 프리즘, 제4회 청룡시리즈어워즈 생중계…"전 세계 325만명 시청" https://n.news.naver.com/mnews/article/138/0002201095
2 조종사노조 "항철위의 조종사 과실 프레임, 법적대응 검토" https://n.news.naver.com/mnews/article/422/0000762597
3 LG '엑사원' B2B 생태계 만든다…"AI 글로벌 톱3 도약할 것" https://n.news.naver.com/mnews/article/011/0004512227
4 여름 휴가 정점 8월 2, 3일 서울~부산·부산~서울 각각  6시간 10분 걸릴 듯 https://n.news.naver.com/mnews/article/658/0000114373
5 대기업 M&A 성과 1위는?…매출 140% 늘어난 '이곳' https://n.news.naver.com/mnews/article/011/0004512171
6 코스피 장중 연고점 찍고 하락…코스닥도 하락 마감 https://n.news.naver.com/mnews/article/056/0011993890
7 한미, 25일 미국서 ‘2+2’ 통상협의…막판 총력전 https://n.news.naver.com/mnews/article/056/0011993755
8 하나證, 퇴직연금 DB형 원리금 보장 상품 2분기 수익률 1위 https://n.news.naver.com/mnews/article/011/0004512120
9 한화에어로, 누리호 기술 이전받는다…25일 항우연과 체결식 https://n.news.naver.com/mnews/article/032/0003384593
10 코로나부터 반도체·관세 파고까지...현대차는 어떻게 버텼나 https://n.news.naver.com/mnews/article/277/0005625841
11 "직장인이 비싼 장어덮밥 어떻게 먹나요"…여기선 1만원도 안 

### 2. Image 다운로드
* referer 요청 헤더를 반드시 설정해야 한다.

In [ ]:
import requests
import os

req_header = {
    'referer':'https://comic.naver.com/webtoon/detail?titleId=783053&no=194&week=tue'      
}

img_urls = [
    'https://image-comic.pstatic.net/webtoon/783053/194/20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_30.jpg',
    'https://image-comic.pstatic.net/webtoon/783053/194/20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_31.jpg',
    'https://image-comic.pstatic.net/webtoon/783053/194/20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_32.jpg'
]

for img_url in img_urls:
    # requests 의 get(url, headers) 함수 호출하기 
    res = requests.get(img_url,headers=req_header)
    print(res.ok, res.status_code) 
    if res.ok:      
        # binary 응답 데이터 가져오기
        img_data = res.content
        # url에서 파일명만 추출하기
        file_name = os.path.basename(img_url)
        # binday data를 file에 write하기
        with open(file_name, 'wb') as file:
            print(f'Writing to {file_name}({len(img_data)})')
            file.write(img_data)


True 200
20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_30.jpg
Writing to 20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_30.jpg(229653)
True 200
20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_31.jpg
Writing to 20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_31.jpg(111107)
True 200
20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_32.jpg
Writing to 20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_32.jpg(93227)


* 현재 요청된 페이지의 image 모두 다운로드 해보기

In [13]:
import requests
from bs4 import BeautifulSoup
import os

webtoon_url = 'https://comic.naver.com/webtoon/detail?titleId=783053&no=194&week=tue'
req_header = {
    'referer':webtoon_url,
}
res = requests.get(webtoon_url)
print(res.status_code)
if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    print(len(soup.select("img[src$='.jpg']")))
    #https://image-comic.pstatic.net/webtoon
    print(len(soup.select("img[src^='https://image-comic.pstatic.net/webtoon783053/194/']")))
    print(len(soup.select("img[src*='IMAG01']")))
    #img src file명을 저장할 list 선언
    img_url_list = list() #['http://xxx/aa.jpg']
    img_tags = soup.select("img[src*='IMAG01']")
    for img_tag in img_tags:
        img_url = img_tag['src']
        img_url_list.append(img_url)
    #print(len(img_url_list),img_url_list[0:2])

    #img 디렉토리가 없으면 생성하기
    imgdir_name = 'img'
    if not os.path.isdir(imgdir_name):
        os.mkdir(imgdir_name)

    for jpg_url in img_url_list[:3]:
        res = requests.get(jpg_url, headers=req_header)
        if res.ok:
            #binary data 추출하기
            img_data = res.content
            # img/xxx.jpg 디렉토리명과 jpg파일명을 join
            dir_name = os.path.join(imgdir_name, os.path.basename(jpg_url))
            with open(dir_name,'wb') as file:
                print(f'Writing to {dir_name}({len(img_data):,} bytes)')
                file.write(img_data)
        else:
           print(f'Error Code = {res.status_code}')

200
335
0
140
Writing to img\20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_1.jpg(143,504 bytes)
Writing to img\20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_2.jpg(134,363 bytes)
Writing to img\20250609204952_371810a746fd64e5cab0a6f546e5e690_IMAG01_3.jpg(117,042 bytes)


### 리팩토링 코드

import requests
from bs4 import BeautifulSoup
import os

# 기본 설정
url = 'https://comic.naver.com/webtoon/detail?titleId=833255&no=3&week=tue'
req_header = {'referer': url}
imgdir_name = 'img'

# 이미지 저장 폴더가 없으면 생성
os.makedirs(imgdir_name, exist_ok=True)

# 웹 페이지 요청 및 확인
res = requests.get(url)
if not res.ok:
    print(f'Error Code = {res.status_code}')
    exit()

# 이미지 URL 추출
soup = BeautifulSoup(res.text, 'html.parser')
img_url_list = [img_tag['src'] for img_tag in soup.select("img[src*='IMAG01']")]

# 이미지 다운로드
for img_url in img_url_list:
    res = requests.get(img_url, headers=req_header)
    if res.ok:
        img_data = res.content
        file_path = os.path.join(imgdir_name, os.path.basename(img_url))
        with open(file_path, 'wb') as file:
            print(f'Writing to {file_path} ({len(img_data):,} bytes)')
            file.write(img_data)
    else:
        print(f'Error Code = {res.status_code} for {img_url}')

### 3. 파일 업로드 하기
* http://httpbin.org/post 업로드 요청을 할 수 있는 url

In [20]:
import requests

upload_files = {
    'img1': open('f1.jpg','rb'),
    'img2': open('f2.jpg','rb')
}
print(upload_files)

url = 'http://httpbin.org/post'
# file 업로드 하려면 requests의 post 함수에 files 속성을 사용합니다.
res = requests.post(url,files=upload_files)
print(res.status_code)
print(type(res.json()))
#print(res.json())
print(res.json()['files']['img1'])


{'img1': <_io.BufferedReader name='f1.jpg'>, 'img2': <_io.BufferedReader name='f2.jpg'>}
200
<class 'dict'>
...


### 4. 캡챠(이미지) API 호출하기
* urllib 사용
* 1. 캡차 키 발급 요청
  2. 캡차 이미지 요청
  3. 사용자 입력값 검증 요청

In [1]:
# 캡차 키 발급 요청
import urllib.request

client_id = "4IHFEzOXgxylfv1jDSPQ" # 개발자센터에서 발급받은 Client ID 값
client_secret = "Nr0uG06CRD" # 개발자센터에서 발급받은 Client Secret 값
code = "0"
url = "https://openapi.naver.com/v1/captcha/nkey?code=" + code
#Request 객체생성
request = urllib.request.Request(url)
#요청해더 설정
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
#서버에 request 전달
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{"key":"SdouYVVLZUTlv0uL"}


In [2]:
# 캡차 이미지 요청
import urllib.request
client_id = "4IHFEzOXgxylfv1jDSPQ" # 개발자센터에서 발급받은 Client ID 값
client_secret = "Nr0uG06CRD" # 개발자센터에서 발급받은 Client Secret 값
key = "SdouYVVLZUTlv0uL" # 캡차 Key 값
url = "https://openapi.naver.com/v1/captcha/ncaptcha.bin?key=" + key
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    print("캡차 이미지 저장")
    response_body = response.read()
    with open('captcha.jpg', 'wb') as f:
        f.write(response_body)
else:
    print("Error Code:" + rescode)

캡차 이미지 저장


In [23]:
#  사용자 입력값 검증 요청
import urllib.request
client_id = "4IHFEzOXgxylfv1jDSPQ" # 개발자센터에서 발급받은 Client ID 값
client_secret = "Nr0uG06CRD" # 개발자센터에서 발급받은 Client Secret 값
code = "1"
key = "jYajuTJxSeKztJQT"
value = "9H7F3X8"
url = "https://openapi.naver.com/v1/captcha/nkey?code=" + code + "&key=" + key + "&value=" + value
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)


{"result":true,"responseTime":180.33}


* requests를 사용하는 코드로 변경하기
* [requests docs](https://requests.readthedocs.io/en/latest/user/quickstart/)

In [3]:
# 사용자 입력값 검증 요청
import os
from dotenv import load_dotenv
import requests

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 값 읽기
client_id = os.getenv("NAVER_CLIENT_ID")  # .env 파일의 NAVER_CLIENT_ID
client_secret = os.getenv("NAVER_CLIENT_SECRET")  # .env 파일의 NAVER_CLIENT_SECRET
key = "SdouYVVLZUTlv0uL"
#os.getenv("NAVER_CAPTCHA_KEY") 
# .env 파일의 NAVER_CAPTCHA_KEY

# 사용자 입력값
code = "1"
value = "HRN8"  # 실제 사용시 사용자 입력값으로 대체 필요

# URL 구성
# query string key1=value1&key2=value2
url = f"https://openapi.naver.com/v1/captcha/nkey?code={code}&key={key}&value={value}"

# 요청 헤더
headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

# 요청 보내기
try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # 4xx, 5xx 오류 발생시 예외 발생
    
    # 응답 처리
    print(response.text)
    
except requests.exceptions.RequestException as e:
    print(f"Error occurred: {e}")
    print(f"Status Code: {response.status_code if 'response' in locals() else 'N/A'}")


{"result":true,"responseTime":313.76}


### 5. 블로그 검색하기

In [9]:
import requests
from pprint import pprint
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 값 읽기
client_id = os.getenv("NAVER_CLIENT_ID")  # .env 파일의 NAVER_CLIENT_ID
client_secret = os.getenv("NAVER_CLIENT_SECRET")  # .env 파일의 NAVER_CLIENT_SECRET

headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret,
}

# query string 문자열을 dict 선언
payload = {
    'query': '파이썬',
    'display': 100,
    'sort': 'date'
}

url = 'https://openapi.naver.com/v1/search/blog.json' #?query=파이썬&display=100&sort=sim

# requests get(url, params, headers) 요청 
res = requests.get(url, params=payload, headers=headers)
# json() 함수로 응답 결과 가져오기
items_data = res.json()['items']
print(type(res.json()))
#pprint(res.json())
print(len(items_data), type(items_data), type(items_data[0]))
pprint(items_data[0])

#[{},{},{}] 
# [] - 100 개의 데이터, {title:'',link:''} - 1개의 블로그 글 데이터


<class 'dict'>
100 <class 'list'> <class 'dict'>
{'bloggerlink': 'blog.naver.com/kevinpanda0801',
 'bloggername': 'kevinpanda0801님의 블로그',
 'description': '',
 'link': 'https://blog.naver.com/kevinpanda0801/223944120302',
 'postdate': '20250723',
 'title': '2025.7.16 <b>파이썬</b>'}


In [12]:
# 'title' , 'bloggername' , 'description' , 'bloggerlink' , 'link'
items_list = list()
item_list = []
for item in items_data:
#     print(item)
    item_list.append(item['title'])
    item_list.append(item['bloggername'])
    item_list.append(item['description'])
    item_list.append(item['bloggerlink'])
    item_list.append(item['link'])
    item_list.append(item['postdate'])

    items_list.append(item_list)
    item_list = []

print(items_list) #[[],[],[]]

# 'data/nhnblog.txt' 파일 생성하기
with open('data/nhnblog.txt','w',encoding="utf-8")as file:
    for items in items_list:
        for item in items:
            item = item + '\n'
            file.write(item)
        file.write('-'*150+'\n')

[['2025.7.16 <b>파이썬</b>', 'kevinpanda0801님의 블로그', '', 'blog.naver.com/kevinpanda0801', 'https://blog.naver.com/kevinpanda0801/223944120302', '20250723'], ['하루 목표 달성률 높이는 체크리스트 작성법', '도형레코드 DH record', "대신 '일주일 안에 <b>파이썬</b> 기본 문법 강의 3개 수강하고 예제 코드 따라 하기'와 같이 현실적인 목표를 설정하는 것이 바람직합니다. 목표 설정 시 현재 자신의 역량 수준을 객관적으로 평가하는 과정이... ", 'blog.naver.com/jigengs', 'https://blog.naver.com/jigengs/223911881591', '20250723'], ['spyder IDE로 python 사용하기', '세상을 가치있게', 'Spyder의 사용은 anacoda 를 설치하여 사용하는 것을 권장: Anaconda 프로그램을 설치하면, Python과 함께 Jupyter Notebook, NumPy, pandas, matplotlib 등 <b>파이썬</b>으로 데이터 사이언스를... ', 'blog.naver.com/gwihyeonseo', 'https://blog.naver.com/gwihyeonseo/223944119740', '20250723'], ['[python] pickle, copy, mask', '세상을 가치있게', 'pickle 모듈은 <b>파이썬</b> 객체를 파일에 저장하거나 네트워크를 통해 전송할 수도 있다. pickle.dump(): <b>파이썬</b> 객체를 파일에 저장하는 데 사용되며, 첫 번째 인자는 저장할 객체, 두 번째 인자는 파일... ', 'blog.naver.com/gwihyeonseo', 'https://blog.naver.com/gwihyeonseo/223944116543', '20250723'], ['<b>파이썬</b> Python 기본 개념 정리표'